# 过拟合与交叉验证

- 训练集与测试集分开
- 怎么分：
    - 存在各种不同切分方案

## 样本切分方案
    

- 基本交叉验证- 随机切分
     - 每次切分都是随机成两个部分。
     
- K折交叉验证
    - 把样本分成K份。每次使用1分作为测试，K-1分作为训练
    - 产生K个分类
    
- 留一交叉验证
    - 1一个作为测试，其余全是训练（用于样本数比较少）
    
- 留p交叉验证
    - p个作为测试吗区域作为训练
    

## sklearn样本切分

### 切分函数

- 仅仅是切分的功能

In [20]:
from sklearn.model_selection import train_test_split
import sklearn.datasets as ds

# 加载数据，
data, target = ds.load_iris(return_X_y=True)
# 切分
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2)

# 数据打乱以后，data与target的索引还是一一对应

### 切分类

- 切分方案

In [21]:
from sklearn.model_selection import train_test_split, KFold
import sklearn.datasets as ds
data, target = ds.load_iris(return_X_y=True)

kf = KFold(n_splits=5, shuffle=True)  # 分成K份以后，随机取一份 
re_split = kf.split(data)   # 返回的是下标
for train, test in re_split:
#     print(sl)
    data_test = data.take(test, axis=0)
#     print(data_test)
    break



### 使用交叉验证来训练与测试决策树

In [38]:
import numpy as np
import sklearn.datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# 加载数据集
data, target = ds.load_iris(return_X_y=True)
# 切分成训练集与测试集
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2)
# 使用训练集训练
classifier = DecisionTreeClassifier()
classifier.fit(data_train, target_train)
# 使用测试集测试（观察效果）
pre = classifier.predict(data_test)
num = len(target_test)
correct_num = (pre == target_test).sum()

print(F'测试样本数：{num}，正确个数：{correct_num}，正确率：{100.0* correct_num/num }%')

测试样本数：30，正确个数：29，正确率：96.66666666666667%


In [40]:
import numpy as np
import sklearn.datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# 加载数据集
data, target = ds.load_iris(return_X_y=True)
classifier = DecisionTreeClassifier()

# 切分成训练集与测试集
kf = KFold(n_splits=10, shuffle=True)  # 分成K份以后，随机取一份 
re_split = kf.split(data)   # 返回的是下标

# 使用训练集训练
for  train_index, test_index in re_split:
    data_train = data.take(train_index, axis=0)
    target_train = target.take(train_index, axis=0)
    
    data_test= data.take(test_index,axis=0)
    target_test = target.take(test_index,axis=0)
    
    classifier.fit(data_train, target_train)
    # 使用测试集测试（观察效果）
    pre = classifier.predict(data_test)
    num = len(target_test)
    correct_num = (pre == target_test).sum()

    print(F'测试样本数：{num}，正确个数：{correct_num}，正确率：{100.0* correct_num/num }%')

测试样本数：15，正确个数：13，正确率：86.66666666666667%
测试样本数：15，正确个数：14，正确率：93.33333333333333%
测试样本数：15，正确个数：14，正确率：93.33333333333333%
测试样本数：15，正确个数：15，正确率：100.0%
测试样本数：15，正确个数：14，正确率：93.33333333333333%
测试样本数：15，正确个数：15，正确率：100.0%
测试样本数：15，正确个数：15，正确率：100.0%
测试样本数：15，正确个数：15，正确率：100.0%
测试样本数：15，正确个数：13，正确率：86.66666666666667%
测试样本数：15，正确个数：15，正确率：100.0%


# KNN近邻算法
    - K-nearest neibourhoods

- 算法: $X = [x_1, x_2, \dots x_n] $，这n个样本属于m个类。存在待分类的样本$x$
    1. 计算$x$到$X$所有样本点的距离$D= [d_1,d_2,\dots,d_n]$
    2. 对$D$排序，取前面K个最小的距离。k的值可以取1-150
    3. 把k个距离对应的样本，统计他们所在的类别。
    4. 那个类别中样本最多，我们认为x属于那个类。

# KNN的算法实现


In [47]:
import numpy as np
import sklearn.datasets as ds

def knn(train_data, train_label, sample,  k):
    
    # 计算待分类的样本到所有训练样本的距离
    diff = ((sample - train_data)**2).sum(axis =1 )
    distances = np.sqrt(diff)   # 每个距离索引对应样本的位置
    #  print(distances)
    # 排序
    sorted_index = distances.argsort()   # 从小到大
    # 取k个最小的,顺便统计每个类的统计数量
    categories = {}  # key=类别，value=所在类别的统计量
    for i in range(k):
        # 取类别
        category = train_label[sorted_index[i]]
        categories[category] = categories.get(category, 0) + 1  # 编程技巧
    # print(categories)
    # 统计k中样本所在类别的数量，数量最大的就是分类
    # 对字典categories按照值排序，取出一个的key就是类别
    sorted_categories = sorted(categories.items(),  key=lambda x: x[1], reverse=True)  # 排序返回列表
    
    return sorted_categories[0][0]


data, target = ds.load_iris(return_X_y=True)
c = knn(data, target, data[50], 20)
c

1

- 找出最佳的K值。